<a href="https://colab.research.google.com/github/pytholabsbot1/cats_dogs_classifier/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
!export KAGGLE_USERNAME=vaqasam
!export KAGGLE_KEY=2ec5d91c66a678bc314920e84c687f3b

In [0]:
!mkdir /root/.kaggle

!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')

63

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 72.9MB/s]
 95% 257M/271M [00:01<00:00, 166MB/s]
100% 271M/271M [00:02<00:00, 140MB/s]
 99% 537M/543M [00:04<00:00, 142MB/s]
100% 543M/543M [00:04<00:00, 130MB/s]


In [0]:

!mkdir /content/train
!mkdir /content/test

!ls /content

sample_data  sampleSubmission.csv  test  test1.zip  train  train.zip


In [0]:
!unzip train.zip -d /content/train

In [0]:
!mkdir /content/train/train/new/
!mkdir /content/train/train/new/dogs
!mkdir /content/train/train/new/cats

!mkdir /content/train/train/new_test/
!mkdir /content/train/train/new_test/dogs
!mkdir /content/train/train/new_test/cats

In [0]:
import os,shutil

for i in os.listdir('/content/train/train'):
  try:
    if('cat' in i):
      shutil.copyfile('/content/train/train/'+i,'/content/train/train/new/cats/'+i)

    else:
      shutil.copyfile('/content/train/train/'+i,'/content/train/train/new/dogs/'+i)
      
  except:
    print('error')
  #print(i)

In [0]:
!ls /content/train/train/new_test/dogs

In [0]:
import os,shutil

c = [i for i in os.listdir('/content/train/train/new/cats')]
d = [i for i in os.listdir('/content/train/train/new/dogs')]

print('length dogs,cats :{} , {}'.format(len(d),len(c)))



for i in range(500):
  os.system('sudo mv /content/train/train/new/dogs/{} /content/train/train/new_test/dogs/{}'.format(d[i],d[i]))
  os.system('sudo mv /content/train/train/new/cats/{} /content/train/train/new_test/cats/{}'.format(c[i],c[i]))

  #print(i)


length dogs,cats :12500 , 12500
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

In [0]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/train/train/new/',
                                                 target_size = (64, 64),
                                                 batch_size = 100,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/train/train/new_test/',
                                            target_size = (64, 64),
                                            batch_size = 100,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 120,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 5)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Found 24500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
120/120 [==============================] - 55s 461ms/step - loss: 0.6926 - acc: 0.5469 - val_loss: 0.6221 - val_acc: 0.6880
Epoch 2/10
120/120 [==============================] - 53s 438ms/step - loss: 0.6245 - acc: 0.6574 - val_loss: 0.7314 - val_acc: 0.5720
Epoch 3/10
120/120 [==============================] - 52s 435ms/step - loss: 0.5802 - acc: 0.6984 - val_loss: 0.5249 - val_acc: 0.7160
Epoch 4/10
120/120 [==============================] - 52s 435ms/step - loss: 0.5451 - acc: 0.7236 - val_loss: 0.2986 - val_acc: 0.8760
Epoch 5/10
120/120 [==============================] - 52s 433ms/step - loss: 0.5196 - acc: 0.7451 - val_loss: 0.4820 - val_acc: 0.7480
Epoch 6/10
120/120 [==============================] - 53s 438ms/step - loss: 0.4995 - acc: 0.7577 - val_loss: 0.4451 - va

In [0]:
!wget https://thenypost.files.wordpress.com/2018/10/102318-dogs-color-determine-disesases-life.jpg

--2019-03-25 13:32:23--  https://thenypost.files.wordpress.com/2018/10/102318-dogs-color-determine-disesases-life.jpg
Resolving thenypost.files.wordpress.com (thenypost.files.wordpress.com)... 192.0.72.3
Connecting to thenypost.files.wordpress.com (thenypost.files.wordpress.com)|192.0.72.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 952719 (930K) [image/jpeg]
Saving to: ‘102318-dogs-color-determine-disesases-life.jpg’

102318-dogs-color-d 100%[===================>] 930.39K  --.-KB/s    in 0.1s    

2019-03-25 13:32:23 (6.56 MB/s) - ‘102318-dogs-color-determine-disesases-life.jpg’ saved [952719/952719]



In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('102318-dogs-color-determine-disesases-life.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)


In [0]:
classifier.predict(test_image)

array([[6.738373e-05]], dtype=float32)

In [0]:
training_set.class_indices

{'cats': 0, 'dogs': 1}